# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f69045d53a0>
├── 'a' --> tensor([[ 1.1653,  0.7952, -0.1437],
│                   [-0.9115,  0.1096, -0.2945]])
└── 'x' --> <FastTreeValue 0x7f690464e9d0>
    └── 'c' --> tensor([[ 1.5585, -1.3677,  0.8756, -1.9274],
                        [-0.2258,  1.0299, -0.3397,  0.6682],
                        [ 0.1623, -0.6909,  0.8866, -0.2236]])

In [4]:
t.a

tensor([[ 1.1653,  0.7952, -0.1437],
        [-0.9115,  0.1096, -0.2945]])

In [5]:
%timeit t.a

58.9 ns ± 0.0778 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f69045d53a0>
├── 'a' --> tensor([[ 0.9301,  1.8045, -0.7907],
│                   [-0.5089, -0.6546, -0.4862]])
└── 'x' --> <FastTreeValue 0x7f690464e9d0>
    └── 'c' --> tensor([[ 1.5585, -1.3677,  0.8756, -1.9274],
                        [-0.2258,  1.0299, -0.3397,  0.6682],
                        [ 0.1623, -0.6909,  0.8866, -0.2236]])

In [7]:
%timeit t.a = new_value

64.1 ns ± 0.0428 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.1653,  0.7952, -0.1437],
               [-0.9115,  0.1096, -0.2945]]),
    x: Batch(
           c: tensor([[ 1.5585, -1.3677,  0.8756, -1.9274],
                      [-0.2258,  1.0299, -0.3397,  0.6682],
                      [ 0.1623, -0.6909,  0.8866, -0.2236]]),
       ),
)

In [10]:
b.a

tensor([[ 1.1653,  0.7952, -0.1437],
        [-0.9115,  0.1096, -0.2945]])

In [11]:
%timeit b.a

54.4 ns ± 0.058 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.3762,  0.7055, -0.6196],
               [-0.7325,  0.5583, -0.6127]]),
    x: Batch(
           c: tensor([[ 1.5585, -1.3677,  0.8756, -1.9274],
                      [-0.2258,  1.0299, -0.3397,  0.6682],
                      [ 0.1623, -0.6909,  0.8866, -0.2236]]),
       ),
)

In [13]:
%timeit b.a = new_value

487 ns ± 0.124 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

872 ns ± 0.315 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.4 µs ± 44.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

139 µs ± 245 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

141 µs ± 626 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f68d8a2d970>
├── 'a' --> tensor([[[ 1.1653,  0.7952, -0.1437],
│                    [-0.9115,  0.1096, -0.2945]],
│           
│                   [[ 1.1653,  0.7952, -0.1437],
│                    [-0.9115,  0.1096, -0.2945]],
│           
│                   [[ 1.1653,  0.7952, -0.1437],
│                    [-0.9115,  0.1096, -0.2945]],
│           
│                   [[ 1.1653,  0.7952, -0.1437],
│                    [-0.9115,  0.1096, -0.2945]],
│           
│                   [[ 1.1653,  0.7952, -0.1437],
│                    [-0.9115,  0.1096, -0.2945]],
│           
│                   [[ 1.1653,  0.7952, -0.1437],
│                    [-0.9115,  0.1096, -0.2945]],
│           
│                   [[ 1.1653,  0.7952, -0.1437],
│                    [-0.9115,  0.1096, -0.2945]],
│           
│                   [[ 1.1653,  0.7952, -0.1437],
│                    [-0.9115,  0.1096, -0.2945]]])
└── 'x' --> <FastTreeValue 0x7f68579e23d0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.5 µs ± 153 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f6857301e50>
├── 'a' --> tensor([[ 1.1653,  0.7952, -0.1437],
│                   [-0.9115,  0.1096, -0.2945],
│                   [ 1.1653,  0.7952, -0.1437],
│                   [-0.9115,  0.1096, -0.2945],
│                   [ 1.1653,  0.7952, -0.1437],
│                   [-0.9115,  0.1096, -0.2945],
│                   [ 1.1653,  0.7952, -0.1437],
│                   [-0.9115,  0.1096, -0.2945],
│                   [ 1.1653,  0.7952, -0.1437],
│                   [-0.9115,  0.1096, -0.2945],
│                   [ 1.1653,  0.7952, -0.1437],
│                   [-0.9115,  0.1096, -0.2945],
│                   [ 1.1653,  0.7952, -0.1437],
│                   [-0.9115,  0.1096, -0.2945],
│                   [ 1.1653,  0.7952, -0.1437],
│                   [-0.9115,  0.1096, -0.2945]])
└── 'x' --> <FastTreeValue 0x7f68573016d0>
    └── 'c' --> tensor([[ 1.5585, -1.3677,  0.8756, -1.9274],
                        [-0.2258,  1.0299, -0.3397,  0.6682],
                 

In [23]:
%timeit t_cat(trees)

29.9 µs ± 88.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

58.4 µs ± 146 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 1.1653,  0.7952, -0.1437],
                [-0.9115,  0.1096, -0.2945]],
       
               [[ 1.1653,  0.7952, -0.1437],
                [-0.9115,  0.1096, -0.2945]],
       
               [[ 1.1653,  0.7952, -0.1437],
                [-0.9115,  0.1096, -0.2945]],
       
               [[ 1.1653,  0.7952, -0.1437],
                [-0.9115,  0.1096, -0.2945]],
       
               [[ 1.1653,  0.7952, -0.1437],
                [-0.9115,  0.1096, -0.2945]],
       
               [[ 1.1653,  0.7952, -0.1437],
                [-0.9115,  0.1096, -0.2945]],
       
               [[ 1.1653,  0.7952, -0.1437],
                [-0.9115,  0.1096, -0.2945]],
       
               [[ 1.1653,  0.7952, -0.1437],
                [-0.9115,  0.1096, -0.2945]]]),
    x: Batch(
           c: tensor([[[ 1.5585, -1.3677,  0.8756, -1.9274],
                       [-0.2258,  1.0299, -0.3397,  0.6682],
                       [ 0.1623, -0.6909,  0.8866, -0.2236]],
         

In [26]:
%timeit Batch.stack(batches)

79.2 µs ± 92 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 1.1653,  0.7952, -0.1437],
               [-0.9115,  0.1096, -0.2945],
               [ 1.1653,  0.7952, -0.1437],
               [-0.9115,  0.1096, -0.2945],
               [ 1.1653,  0.7952, -0.1437],
               [-0.9115,  0.1096, -0.2945],
               [ 1.1653,  0.7952, -0.1437],
               [-0.9115,  0.1096, -0.2945],
               [ 1.1653,  0.7952, -0.1437],
               [-0.9115,  0.1096, -0.2945],
               [ 1.1653,  0.7952, -0.1437],
               [-0.9115,  0.1096, -0.2945],
               [ 1.1653,  0.7952, -0.1437],
               [-0.9115,  0.1096, -0.2945],
               [ 1.1653,  0.7952, -0.1437],
               [-0.9115,  0.1096, -0.2945]]),
    x: Batch(
           c: tensor([[ 1.5585, -1.3677,  0.8756, -1.9274],
                      [-0.2258,  1.0299, -0.3397,  0.6682],
                      [ 0.1623, -0.6909,  0.8866, -0.2236],
                      [ 1.5585, -1.3677,  0.8756, -1.9274],
                      [-0.2258,  

In [28]:
%timeit Batch.cat(batches)

146 µs ± 330 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

307 µs ± 1.81 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
